# 示例程序以美元为例
## 初始化
随机生成矢量，对应属性 variable - country, capital, currency；值 value 包括 US Mexico

In [1]:
import torch, torchhd
d = 10000
keys = torchhd.random(3, d)
country, capital, currency = keys

usa, mex = torchhd.random(2, d)  # United States and Mexico
wdc, mxc = torchhd.random(2, d)  # Washington D.C. and Mexico City
usd, mxn = torchhd.random(2, d)  # US Dollar and Mexican Peso
usa.shape, usa

(torch.Size([10000]), MAPTensor([-1.,  1.,  1.,  ..., -1.,  1.,  1.]))

## Mapping between analogical structure
将 variable 和 value 绑定起来，即生成 US MX 的全息向量
$$
US = (country * usa + capital * wdc + currency * usd)
$$
These hypervectors are then combined to make a single hypervector for each country using a hash table structure. A hash table encodes key-value pairs as: $k1 * v1 + k2 * v2 + ... + kn * vn$.

In [2]:
us_values = torch.stack([usa, wdc, usd])
us = torchhd.hash_table(keys, us_values)
us

MAPTensor([-1.,  1., -1.,  ...,  1., -1.,  1.])

In [3]:
mx_values = torch.stack([mex, mxc, mxn])
mx = torchhd.hash_table(keys, mx_values)
mx

MAPTensor([-1., -1.,  1.,  ..., -3.,  3., -3.])

将两个全息向量组合
$$
F_{UM} = US * MX\\
= USA * MEX + WDC * MXC + DOL * PES + noise
$$

In [9]:
# combine all the associated information
mx_us = torchhd.bind(torchhd.inverse(us), mx)
mx_us

MAPTensor([ 3., -9.,  1.,  ..., -1., -3.,  1.])

通过 US 和 MX 的映射关系，推导 usd 的对应
$$
DOL * F_{UM} = PES + noise
$$

In [10]:
# query for the dollar of mexico
usd_of_mex = torchhd.bind(mx_us, usd)
usd_of_mex

MAPTensor([ 3., -9., -1.,  ..., -1., -3., -1.])

memory

In [11]:
memory = torch.cat([keys, us_values, mx_values], dim = 0)
memory, memory.shape

(MAPTensor([[ 1., -1.,  1.,  ..., -1.,  1., -1.],
            [-1.,  1., -1.,  ...,  1., -1., -1.],
            [ 1., -1.,  1.,  ..., -1., -1., -1.],
            ...,
            [ 1., -1., -1.,  ..., -1.,  1., -1.],
            [-1.,  1., -1.,  ...,  1., -1., -1.],
            [ 1., -1., -1.,  ...,  1.,  1.,  1.]]),
 torch.Size([9, 10000]))

query 的 hv 和 mxn (0.3349)二者最为相似

In [12]:
torchhd.cosine_similarity(usd_of_mex, memory)
# The hypervector for the Mexican Peso is the most similar.

MAPTensor([-0.0043,  0.0019,  0.0064, -0.0279,  0.0018, -0.0028, -0.0038,
            0.0143,  0.3349])

## Programming Notes

在PyTorch中，`torch.stack`和`torch.cat`是两个用于沿新维度合并张量的函数，但它们的用法和目的有所不同。

### torch.stack

`torch.stack`函数会将一系列张量沿着一个新的维度堆叠起来。与`torch.cat`不同，`torch.stack`会增加一个新的维度，并且要求所有输入张量的形状完全相同。

**用法**:
```python
torch.stack(tensors, dim=0, *, out=None) → Tensor
```
- `tensors`：一个张量序列，它们的形状必须完全相同。
- `dim`：新维度插入的位置。
- `out`：一个可选的张量，用于存储输出。

In [13]:
import torch

tensor1 = torch.tensor([1, 2])
tensor2 = torch.tensor([3, 4])

stacked_tensor = torch.stack((tensor1, tensor2))  # 默认在第0维堆叠
stacked_tensor

tensor([[1, 2],
        [3, 4]])

### torch.cat

`torch.cat`函数会将一系列张量沿着一个已存在的维度连接起来。所有输入张量的形状必须相同，除了拼接的维度。

**用法**:
```python
torch.cat(tensors, dim=0, *, out=None) → Tensor
```
- `tensors`：一个张量序列，它们的形状必须在非拼接维度上相同。
- `dim`：拼接的维度。
- `out`：一个可选的张量，用于存储输出。

在上述示例中，`torch.stack`创建了一个新的维度，并将两个张量堆叠在一起，而`torch.cat`则将两个张量沿着第0维（行）拼接在一起。

总结来说，`torch.stack`用于创建一个新的维度并堆叠张量，而`torch.cat`用于在一个已存在的维度上拼接张量。


In [14]:
import torch

tensor1 = torch.tensor([[1, 2], [3, 4]])
tensor2 = torch.tensor([[5, 6], [7, 8]])

concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)  # 在第0维拼接
concatenated_tensor

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

In [15]:
torchhd.cosine_similarity(usa, mex)

MAPTensor(-0.0048)